### 특정 웹툰 페이지의 모든 image를 다운로드 받기
* soup.select("img[src$='jpg']")
* img 폴더를 생성하고 그 아래에 파일을 저장하기

In [12]:
import requests
from bs4 import BeautifulSoup
import re

main_url = 'https://comic.naver.com/webtoon/detail.nhn?titleId=712362&no=94&amp;weekday=fri'
res = requests.get(main_url)
print(res.status_code)
if res.ok:
    html = res.text
    soup = BeautifulSoup(html,'html.parser')
    
    print(len(soup.select("img[src$='.jpg']")))
    img_url_list = []
    for img_url in soup.select("img[src$='.jpg']"):
        #print(type(img_url),img_url)
        # img 태그의 src 속성의 값을 가져와서 img_url_list에 추가하기
        img_url_list.append(img_url['src'])

print(len(img_url_list))        
img_url_list[0:3]   

200
0
0


[]

In [13]:
import os
#img 디렉토리 생성
#img 디렉토리가 없으면
if not os.path.isdir('img'):
    os.mkdir('img')    

In [14]:
#image_url_list에 있는 url로 요청해서 image들을 다운로드 하기
for idx,img_url in enumerate(img_url_list,1):
    print(f'---> 다운로드 번호 {idx}')
    req_header = {
        'referer':main_url
    }
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_name = os.path.basename(img_url)
        with open('img/'+file_name,'wb') as file:
            print(f'파일명 :{file_name} ({len(img_data)}) bytes')
            file.write(img_data)

### image 다운로드
* 위의 코드를 함수로 정리
* os.mkdir와 os.makedirs 
    - mkdir은 한 폴더만 생성이 가능
    - makedirs는 './a/b/c' 처럼 Sub 디렉토리를 생성이 가능

In [15]:
def download_image(title,url):
    import requests
    from bs4 import BeautifulSoup
    import os
    
    req_header = {
        'referer': url
    }
    
    res = requests.get(url)
    print(res.ok)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        imgurl_list = []
        print(len(soup.select("img[src*='IMAG01']")))
        for img_tag in soup.select("img[src*='IMAG01']"):
            imgurl_list.append(img_tag['src'])
            #print(img_tag)
                
        #image를 저장할 디렉토리 생성
        
        #dir_name = 'img/' + title
        dir_name = os.path.join('img',title)
        print(dir_name)
        if not os.path.isdir(dir_name):
            os.makedirs(dir_name)
        
        for idx,img_url in enumerate(imgurl_list,1):
            print(idx, img_url)
            res = requests.get(img_url,headers=req_header)
            #print(res.status_code)
            if res.ok:
                img_data = res.content
                #print(len(img_data))
                file_name = os.path.basename(img_url)
                #print(file_name)
                file_name = dir_name + '/' + file_name
                with open(file_name,'wb') as file:
                    print(f'{file_name} (파일크기 : {len(img_data)}))')
                    file.write(img_data)
                
                
download_image('낢이사는이야기','https://comic.naver.com/webtoon/detail?titleId=833255&no=34&week=tue')

True
0
img\낢이사는이야기


#### 네이버 웹툰 홈에서 추천웹툰의 제목과 url 를 알아내기

In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

main_url = 'https://comic.naver.com/index.nhn'

res = requests.get(main_url)
if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')
    print(len(soup.select('div.genreRecomInfo2')))
    
    recom_webtoon_list = []
    for div_tag in soup.select('div.genreRecomInfo2'):
        recom_webtoon_dict = {}
        a_tag1 = div_tag.select('h6 a')[0]
        #웹툰제목
        title = a_tag1.text.strip()
        #웹툰의 url
        webtoon_url=urljoin(main_url, a_tag1['href'])
        
        #웹툰의 회차의 url
        a_tag2 = div_tag.select('p a')[0]
        webtoon_episode_url = urljoin(main_url,a_tag2['href'])
                
        recom_webtoon_dict['title'] = title
        recom_webtoon_dict['webtoon_url'] = webtoon_url
        recom_webtoon_dict['episode_url'] = webtoon_episode_url
        
        recom_webtoon_list.append(recom_webtoon_dict)
        
    print(recom_webtoon_list)                

0
[]


#### 6개의 만화의 제목과 회차url을 꺼내서 download_image() 함수를 호출한다.

In [17]:
for idx,webtoon in enumerate(recom_webtoon_list,1):
    print('---->>> ', idx, webtoon['title'])
    download_image(webtoon['title'],webtoon['episode_url'])